In [12]:

import sys
import os
from os import path
from os import walk

import re

import pandas as pd
import json
import logging
import pandas_profiling
from pandas.io.json import json_normalize 

## Get Columns

In [45]:
with open("../data/fhir-json/2rows/DiagnosticReport/1.DiagnosticReport.json") as f:
    d = json.load(f)

awhcuris = json_normalize(d)
address_data = awhcuris.head(31)
address_data.T

data_list = list(address_data)
data_list

['code.coding',
 'code.text',
 'context.reference',
 'effectiveDateTime',
 'id',
 'issued',
 'patientId',
 'resourceType',
 'result',
 'status',
 'subject.reference']

## Flatten

In [28]:
def removekey(d, key):
    r = dict(d)
    print('r: ', r)
    del r[key]
    print('after del r[key]: ', r)
    return r

def flattenDict(d, result=None):
    if result is None:
        result = {}
        
    for key, value in list(d.items()):
        value = d[key]

        #CASSANDRA MAPPING
            
        if isinstance(value, dict):
        #if d['type'] =='object':
            value1 = {}
            for keyIn in value:
                value1[".".join([key,keyIn])]=value[keyIn]
            flattenDict(value1, result)
        elif isinstance(value, (list, tuple)):   
            for indexB, element in enumerate(value):
                if isinstance(element, dict):
                    value1 = {}
                    index = 0
                    for keyIn in element:
                        newkey = ".".join([key,keyIn])        
                        value1[".".join([key,keyIn])]=value[indexB][keyIn]
                        index += 1
                    for keyA in value1:
                        flattenDict(value1, result)   
        else:
            result[key]=value
    
    return result

In [15]:
class MakeDirectory:
    
    def __init__(self, path):
        self.path = path

    def create_dir(self, folders=None):
        dirName = ''
        
        if isinstance(folders, list):
            
            for folder in folders:
                dirName =  self.path + '/' + folder
                try:
                    os.makedirs(dirName)
                    print("Directory " , dirName ,  " Created ") 
                except FileExistsError:
                    print("Directory " , dirName ,  " already exists")
        else:
            os.makedirs(dirName)

## Folder directory Iterator

In [39]:
import re

from os import path
from os import walk

class FileIterator:
    
    def __init__(self, input_root, input_folders):
        self.input_data_dir = input_root
        self.input_folders = input_folders
        
    def iterate_filenames(self):
        resources = self.__iterate_dirfiles()
        files = self.__iterate_file(resources)
        return files
        
    def __iterate_dirfiles(self):
        path_files = []
        file_names = self.input_folders

        for file in file_names:
            path_files.append(input_data_dir + '/' + file)
        return path_files
    
    def __iterate_file(self, path_dbs):
        path_db_workload_files = []
        
        for path_db in path_dbs:
            
            for root, dirs, files in walk(path_db):  
                
                for filename in files:
                    print('filename: ', filename)
                    print('path_db_workload_files: ', root + '/' + filename)
                    path_db_workload_files.append(root + '/' + filename)
                    #if ''.join(dirs) != '.ipynb_checkpoints':
                        #pass
                    #else: 
                        #if ".swp" not in filename and ".DS_Store" not in filename and ".txt~" not in filename:
                        #path_db_workload_files.append(root + '/' + filename)
                        
        
        return path_db_workload_files

## Filecontent reader

In [17]:
class ContentReader():
    
    def __init__(self, filename_paths):
        self.filename_paths = filename_paths
        
    def get_data(self):
        data = []
        for filename_path in self.filename_paths:
            
            json = ReadJSON()
            data.append(json.read(filename_path))

        return data

## File Open

In [1]:
class ReadJSON():
    
    @staticmethod
    def read(file_path):
        with open(file_path) as f:
            d = json.load(f)
        return flattenDict(d)

## Print Flatten FHIR JSON

In [19]:
class MakeResult():
    def __init__(self, data, source_paths, destination_path):
        self.source_paths = source_paths
        self.data = data
        self.destination_path = destination_path
        
    def outputs(self):      
        for resource in self.data:
           
            destination_path = self.destination_path + '/' + resource['resourceType']
            #print('destination: ' ,destination_path)
            id = resource['id'].split('-')[0]
     
            f = open(destination_path + '/' + id + '.' + resource['resourceType'] + '.json', 'w')
            f.write(json.dumps(resource))
            f.close()
        return

In [41]:
root_dir = '../'

#input_data_folders = ['Condition']
#input_data_dir = root_dir + 'data/fhir-json/3rows'

input_data_folders = ['DiagnosticReport']
input_data_dir = root_dir + 'data/fhir-json/2rows'

output_directory = root_dir + 'data/fhir-json/Cassandra'

directory = MakeDirectory(output_directory)
directory.create_dir(input_data_folders)
            
files = FileIterator(input_data_dir, input_data_folders)
input_dirpaths = files.iterate_filenames()

data = []
files = ContentReader(input_dirpaths)
data = files.get_data()

result = MakeResult(data, input_dirpaths, output_directory)
result.outputs()

Directory  ../data/fhir-json/Cassandra/DiagnosticReport  Created 
filename:  0.DiagnosticReport.json
path_db_workload_files:  ../data/fhir-json/2rows/DiagnosticReport/0.DiagnosticReport.json
filename:  1.DiagnosticReport.json
path_db_workload_files:  ../data/fhir-json/2rows/DiagnosticReport/1.DiagnosticReport.json
filename:  2.DiagnosticReport.json
path_db_workload_files:  ../data/fhir-json/2rows/DiagnosticReport/2.DiagnosticReport.json
